# Supervised Learning with Decision Trees

In [1]:
import os
from PIL import Image
import numpy as np 

#Dataset Directory 
alphabet_dir = "ASL_Dataset/Train"

images = []
labels = []

In [2]:
max_images_per_folder = 100

for folder_name in os.listdir(alphabet_dir):
    folder_path = os.path.join(alphabet_dir, folder_name)

    image_counter = 0

    for filename in os.listdir(folder_path):
        if filename.endswith(".jpg") or filename.endswith(".png"):
            if image_counter >= max_images_per_folder:
                break
            image_path = os.path.join(folder_path, filename)
            image = Image.open(image_path)

            # Resize the image to 256x256 pixels
            image = image.resize((256, 256))

            image = np.array(image)
            images.append(image)
            label = folder_name
            labels.append(label)
            image_counter += 1

images = np.array(images)
labels = np.array(labels)

In [3]:
len(labels)

2800

In [4]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

In [5]:
X_train, X_rem, y_train, y_rem = train_test_split(images, labels, test_size=0.4, random_state=2411)

X_valid, X_test, y_valid, y_test = train_test_split(X_rem, y_rem, test_size=0.5)

X_train = X_train.reshape(X_train.shape[0], -1)
X_test = X_test.reshape(X_test.shape[0], -1)
X_valid = X_valid.reshape(X_valid.shape[0], -1)



In [7]:
for max_d in range(1,16):
    classifier = DecisionTreeClassifier(max_depth=max_d, random_state=2411)

    classifier.fit(X_train, y_train)
    y_pred = classifier.predict(X_test)

    print('The Training Accuracy for max_depth {} is:'.format(max_d), classifier.score(X_train, y_train))
    print('The Validation Accuracy for max_depth {} is:'.format(max_d), classifier.score(X_valid, y_valid))
    print('')


The Training Accuracy for max_depth 1 is: 0.07380952380952381
The Validation Accuracy for max_depth 1 is: 0.04642857142857143

The Training Accuracy for max_depth 2 is: 0.11547619047619048
The Validation Accuracy for max_depth 2 is: 0.08214285714285714

The Training Accuracy for max_depth 3 is: 0.1767857142857143
The Validation Accuracy for max_depth 3 is: 0.12857142857142856

The Training Accuracy for max_depth 4 is: 0.25595238095238093
The Validation Accuracy for max_depth 4 is: 0.21071428571428572

The Training Accuracy for max_depth 5 is: 0.3380952380952381
The Validation Accuracy for max_depth 5 is: 0.26071428571428573

The Training Accuracy for max_depth 6 is: 0.3904761904761905
The Validation Accuracy for max_depth 6 is: 0.2857142857142857

The Training Accuracy for max_depth 7 is: 0.46130952380952384
The Validation Accuracy for max_depth 7 is: 0.3125

The Training Accuracy for max_depth 8 is: 0.5190476190476191
The Validation Accuracy for max_depth 8 is: 0.35892857142857143

Th

In [29]:
classifier.tree_.max_depth

31